# Test Valuation Engines

This notebook walks through:

1. Loading historical SOFR fixings
2. Building a simple two-index yield curve
3. Valuing various products (bullet, IBOR cashflow, OIS cashflow, futures, swaps)

---

## 1. Imports & Setup

In [1]:
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor
from fixedincomelib.diagnostics import check_calibration, assert_calibrated

## 2. Load SOFR Fixings

In [2]:
IndexManager.instance()

## 3. Build Dummy Yield Curve

In [3]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [4]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [5]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"
}]


In [6]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

['SOFR-1B']
Curve components: dict_keys(['SOFR-1B'])


In [7]:
from fixedincomelib.date import addPeriod, moveToBusinessDay

StartDate = "2025-09-24"
print(Date(StartDate).ISO(),yc.forward(index="SOFR-1B", effectiveDate=Date(StartDate),termOrTerminationDate="3M"))
for i in range(20):
    new_date = addPeriod(StartDate,"3M")
    print(new_date.ISO(), 
          yc.forward(index="SOFR-1B", effectiveDate=new_date,termOrTerminationDate="3M"),
          yc.discountFactor(index='SOFR-1B', to_date=new_date))
    StartDate = moveToBusinessDay(input_date=new_date,biz_conv="F",hol_conv="USGS")
    

2025-09-24 0.04299999999999973
2025-12-24 0.0420000000000007 0.9892474300176142
2026-03-24 0.04099999999999925 0.9789682632534528
2026-06-24 0.039999999999999654 0.9688172118009166
2026-09-24 0.03919999999999943 0.9590139580079465
2026-12-24 0.03840000000000021 0.9496044331910601
2027-03-24 0.03759999999999957 0.9405749140165017
2027-06-24 0.03680000000000041 0.9316230516270895
2027-09-24 0.036198974472346455 0.9229432827986365
2027-12-24 0.035600000000000444 0.9145744187699917
2028-03-24 0.03500914087246375 0.9064176668986448
2028-06-24 0.03445517607961423 0.8983821377774136
2028-09-26 0.029846677637727997 0.8903818949851416
2028-12-26 0.030178307389258308 0.8837146419577939
2029-03-26 0.02952225722862226 0.877097313870723
2029-06-26 0.029522257228623127 0.8705295368818605
2029-09-26 0.034826580828682334 0.8640109399484984
2029-12-26 0.03521354283788991 0.8564710945541244
2030-03-26 0.03444803103706622 0.848997046091181
2030-06-26 0.03444803103706622 0.8415882203786393
2030-09-26 0.03

In [8]:
comp = yc.retrieveComponent("SOFR-1B")

In [9]:
yc.forwardRateGradientWrtModelParameters(index="SOFR-1B", start_time= "2025-09-24", end_time="2026-02-24", scaler =1, accumulate=False)

In [10]:
yc.gradient_

array([0.61075992, 0.40717328, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [11]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B", to_date="2045-12-24", scaler =1, accumulate=False)

In [12]:
yc.gradient_

array([-0.10909254, -0.10909254, -0.10909254, -0.10909254, -0.10909254,
       -0.10909254, -0.10909254, -0.10909254, -0.10909254, -0.10909254,
       -0.10909254, -0.10909254, -0.43879442, -0.43637015, -0.43637015,
       -0.43758229, -0.43515801, -0.43637015, -0.43637015, -2.18185073,
       -2.18185073, -0.10666826, -0.        , -0.        , -0.        ,
       -0.        ])

In [13]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B", to_date="2045-12-24", scaler =-1, accumulate=True)

In [14]:
yc.gradient_

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
comp.state_variables


[0.043243186519581825,
 0.04178103144615489,
 0.04169306522517946,
 0.04068131461630934,
 0.039440471052749955,
 0.03821685121924658,
 0.0382520685320776,
 0.0374419918802089,
 0.036435771528610085,
 0.03583456068222943,
 0.03561871936078645,
 0.03511210724602266,
 0.03006503547864423,
 0.035059447352288414,
 0.04019007362698685,
 0.039332253743042195,
 0.04258954406742226,
 0.04089405997116263,
 0.04160884120252573,
 0.041694364677067663,
 0.04475533979347685,
 0.045740020759203685,
 0.04855696932963484,
 0.04508161016568755,
 0.0493920830586393,
 0.052719152963869725]

In [16]:
comp.pillar_dates


[Date(24,12,2025),
 Date(24,3,2026),
 Date(24,6,2026),
 Date(24,9,2026),
 Date(24,12,2026),
 Date(24,3,2027),
 Date(24,6,2027),
 Date(24,9,2027),
 Date(24,12,2027),
 Date(24,3,2028),
 Date(24,6,2028),
 Date(24,9,2028),
 Date(26,9,2029),
 Date(26,9,2030),
 Date(26,9,2031),
 Date(27,9,2032),
 Date(26,9,2033),
 Date(26,9,2034),
 Date(26,9,2035),
 Date(26,9,2040),
 Date(26,9,2045),
 Date(26,9,2050),
 Date(27,9,2055),
 Date(28,9,2065),
 Date(26,9,2075),
 Date(26,9,2085)]

In [17]:
comp.pillar_times_to_date


[0.25,
 0.5,
 0.75,
 1.0,
 1.25,
 1.5,
 1.75,
 2.0,
 2.25,
 2.5,
 2.75,
 3.0,
 4.0055555555555555,
 5.0055555555555555,
 6.0055555555555555,
 7.008333333333334,
 8.005555555555556,
 9.005555555555556,
 10.005555555555556,
 15.005555555555556,
 20.005555555555556,
 25.005555555555556,
 30.008333333333333,
 40.01111111111111,
 50.00555555555555,
 60.00555555555555]

In [18]:
comp.pillar_nodes

[PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M September 24th, 2025 x December 24th, 2025', pillar_index=0, pillar_time=0.25, pillar_date=Date(24,12,2025), start_date=Date(24,9,2025), end_date=Date(24,12,2025), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object at 0x0000023F37F0B980>, state_value=0.043243186519581825),
 PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 x March 24th, 2026', pillar_index=1, pillar_time=0.5, pillar_date=Date(24,3,2026), start_date=Date(24,12,2025), end_date=Date(24,3,2026), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object at 0x0000023F37F34D70>, state_value=0.04178103144615489),
 PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x June 24th, 2026', pillar_index=2, pillar_time=0.75, pillar_date=Date(24,6,2026), start_date=Date(24,3,2026), end_date=Date(24,6,2026), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object at 0x0000023F37F35010>, state_value=0.041

In [19]:
check_calibration(comp, verbose=True)


RFR FUTURE SOFR-FUTURE-3M September 24th, 2025 x December 24th, 2025  PV=-1.405805e-14  Fut model=95.7000  mkt=95.7000  price_error=+2.8422e-14
RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 x March 24th, 2026  PV=-6.955988e-14  Fut model=95.8000  mkt=95.8000  price_error=-7.1054e-14
RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x June 24th, 2026  PV=+4.130316e-14  Fut model=95.9000  mkt=95.9000  price_error=+7.1054e-14
RFR FUTURE SOFR-FUTURE-3M June 24th, 2026 x September 24th, 2026  PV=+0.000000e+00  Fut model=96.0000  mkt=96.0000  price_error=+2.8422e-14
RFR FUTURE SOFR-FUTURE-3M September 24th, 2026 x December 24th, 2026  PV=+6.747345e-14  Fut model=96.0800  mkt=96.0800  price_error=+5.6843e-14
RFR FUTURE SOFR-FUTURE-3M December 24th, 2026 x March 24th, 2027  PV=-1.336637e-14  Fut model=96.1600  mkt=96.1600  price_error=-1.4211e-14
RFR FUTURE SOFR-FUTURE-3M March 24th, 2027 x June 24th, 2027  PV=+6.619580e-14  Fut model=96.2400  mkt=96.2400  price_error=+5.6843e-14
RFR FUTURE SOFR-FUTU

6.955987879944222e-14

In [20]:
yc.jacobian()

array([[-100.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,  -98.924743  ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,  -97.89682633,    0.        ,
           0.        ,    0.        , 

In [21]:
yc._jacobian_row_labels

['RFR FUTURE SOFR-FUTURE-3M September 24th, 2025 x December 24th, 2025',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 x March 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x June 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2026 x September 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M September 24th, 2026 x December 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2026 x March 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2027 x June 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2027 x September 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M September 24th, 2027 x December 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2027 x March 24th, 2028',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2028 x June 24th, 2028',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2028 x September 24th, 2028',
 'RFR SWAP USD-SOFR-OIS September 26th, 2029',
 'RFR SWAP USD-SOFR-OIS September 26th, 2030',
 'RFR SWAP USD-SOFR-OIS September 26th, 2031',
 'RFR SWAP USD-SOFR-OIS Septemb


## 4. Value Bullet Cashflow


In [22]:
# Bullet cashflow PV
bullet = ProductBulletCashflow("2028-09-24", "USD", 1000000, "LONG")
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    bullet
)
ve.calculateValue()
print("Bullet CF PV:", ve.value)

Bullet CF PV: ['USD', np.float64(890530.626090714)]


## 5. Value IBOR Cashflow

In [23]:
ibor_cf = ProductIborCashflow(
    startDate="2025-09-27",
    endDate="2025-12-27",
    index="USD-LIBOR-BBA-3M",
    spread=0.0,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_cf)
ve.calculateValue()
print("Ibor CF PV:", ve.value)

AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

## 6. Value OIS Cashflow

In [ ]:
ois_cf = ProductOvernightIndexCashflow(
    effectiveDate="2025-09-24",
    termOrEnd="6M",
    index="SOFR-1B",
    compounding="COMPOUND",
    spread=0.0,
    notional=1000000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    ois_cf
)
ve.calculateValue()
print("OIS CF PV:", ve.value)

OIS CF PV: ['USD', np.float64(21364.87983425428)]


## 7. Value Futures

In [ ]:
# IBOR Future PV
future = ProductFuture(
    effectiveDate="2025-06-27",
    index="USD-LIBOR-BBA-3M",
    strike=99.5,
    notional=1_000_000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {"FUNDING INDEX": "SOFR-1B"}, future)
ve.calculateValue()
print("Future PV:", ve.value)

AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

In [ ]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="AVERAGE",
    strike=95.9,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

RFR Future PV: ['USD', np.float64(197849.48600349753)]


In [ ]:
ve.calculateFirstOrderRisk()
print(ve.firstOrderRisk_)

[99950537.62849912        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


## 8. Value a Portfolio

In [ ]:
# Simple portfolio of bullet + IBOR CF
portfolio = ProductPortfolio([bullet, ois_cf], weights=[0.5, 0.5])
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    portfolio
)
ve.calculateValue()
print("Portfolio PV:", ve.value)

Portfolio PV: ['USD', np.float64(911895.5059249683)]


## 9. Value Swaps (IBOR & OIS)

In [ ]:
# IBOR Swap
ibor_swap = ProductIborSwap(
    effectiveDate="2025-06-27",
    maturityDate="2026-06-27",
    frequency="6M",
    iborIndex="USD-LIBOR-BBA-3M",
    spread=0.0,
    fixedRate=0.015,
    notional=1_000_000,
    position="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_swap
)
ve.calculateValue()
print("IBOR Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())



AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

In [24]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-09-24",
    maturityDate="2045-09-24",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.045,
    notional=1000000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(67288.56736169301)] Par Rate: 0.04010054191647455


In [25]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk_

array([ -266822.14184042,  -266822.14184042,  -261286.39599948,
        -261286.39599948,  -255783.49517456,  -255783.49517456,
        -250467.29299042,  -250467.29299042,  -245179.91294197,
        -245179.91294197,  -240035.91352198,  -240035.91352198,
        -934861.91659779,  -890416.62057345,  -852487.23882619,
        -818125.04906681,  -778709.45625644,  -747095.80010476,
        -714807.1201782 , -3133578.45271143, -2502040.62248144,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [26]:
from fixedincomelib.utilities import createValueReport

pv_only = createValueReport({"FUNDING INDEX": "SOFR-1B"}, yc, ois_swap, request="value")

In [ ]:
print(pv_only)